Run the config file to authenticate script and query Key Vault

Second script. Run after 'Query_Activites'

In [39]:
%run config

#to see if scope is there

#dbutils.secrets.listScopes()
client_id = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientid") 

client_secret = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientsecret") 

new_refresh_token = dbutils.secrets.get(scope = "key_vault_secrets", key = "newrefreshtoken")

activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "activityidpath") 

historical_activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "historicalactivitydfpath") 

segment_effort_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "segmenteffortpath") 

import requests

import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)



auth_url = "https://www.strava.com/oauth/token"

activites_url = "https://www.strava.com/api/v3/athlete/activities"





payload = {

    'client_id':  client_id,

    'client_secret': client_secret,

    'refresh_token': new_refresh_token,

    'grant_type': 'refr

Dependencies 

In [40]:
from pyspark.sql.functions import * 
import pandas as pd

### API Query to get more specific details for each activity, need to pass each activity off individually 

Get full activity dataset from what is written in storage, should be all activites

In [41]:

full_activity_dataset = spark.read.format("delta").load(historical_activity_id_path)

In [42]:
full_activity_dataset.show()

+------------+--------------------+-------------------+--------+-----------+------------+----------+--------------------+--------------------+--------------------+
|activity_ids|          start_date|      activity_name|distance|moving_time|elapsed_time|sport_type|total_elevation_gain|    ingest_file_name|         ingested_at|
+------------+--------------------+-------------------+--------+-----------+------------+----------+--------------------+--------------------+--------------------+
|  7480841356|2022-07-16T22:26:16Z|          city wide| 32228.3|       5488|        6756|      Ride|               375.5|activity_information|2022-09-19 21:37:...|
|  7454211244|2022-07-12T01:28:02Z|       straight up |  1710.1|       1297|        1315|      Walk|               142.8|activity_information|2022-09-19 21:37:...|
|  7437771392|2022-07-09T00:06:28Z|        friday ride| 23022.7|       4455|        5896|      Ride|               296.4|activity_information|2022-09-19 21:37:...|
|  7411376407|20

In [48]:
full_activity_dataset.select('ingested_at').distinct().show()

+--------------------+
|         ingested_at|
+--------------------+
|2022-09-19 21:37:...|
|2022-11-14 22:29:...|
|2022-10-26 17:40:...|
|2022-10-17 17:59:...|
+--------------------+

Ggrab all of the disinct activity IDs 

In [43]:
full_activity_ids = full_activity_dataset.select('activity_ids').distinct().rdd.flatMap(lambda x: x).collect()

Take unique activity ids, and extract all of the segments associated with those activities

In [35]:
def query_segments(activity_ids):
    """Gets all segment_ids for each activity_id submitted
    Returns distinct values"""
    df = pd.DataFrame()
    activity_id_list =[]
    segment_id_list =[]
    for id in activity_ids:
        activity_id_urls = ("{}{}?include_all_efforts= True").format("https://www.strava.com/api/v3/activities/",id)
        header = {'Authorization': 'Bearer ' + access_token}
        param = {'per_page': 200, 'page': 1}
        my_activity = requests.get(activity_id_urls, headers=header, params=param).json()

        segment_effort_count =  len(my_activity['segment_efforts'])
        count = 0
        while count < segment_effort_count:

            activity_id = my_activity['segment_efforts'][count]['activity']['id']
            segment_id = my_activity['segment_efforts'][count]['id']
            activity_id_list.append(activity_id)
            segment_id_list.append(segment_id)
                  
            columns = ['segment_id', 'activity_id']
            extracted_data = [segment_id_list, activity_id_list]
            segment_df = pd.DataFrame.from_dict(dict(zip(columns, extracted_data)))

            df= df.append(segment_df)
            count += 1

    #convert pandas df to spark
        
    segment_spark_df = spark.createDataFrame(df)

    #drop duplicate entries
    segment_spark_df = segment_spark_df.dropDuplicates()

    segment_spark_df = segment_spark_df.select(concat(segment_spark_df.segment_id,segment_spark_df.activity_id).alias("Activity_Segment_JointID"), 'segment_id','activity_id')

    segment_spark_df = segment_spark_df.withColumn("ingest_file_name", lit("segment_efforts_ids")) \
                                .withColumn("ingested_at", lit(current_timestamp()))

    return segment_spark_df

Need to compare activites already stored with segments as there are limits for strava API

Initially will go 99 request, but might need to reduce that to save requests for segments

Query all segments from all activities 

In [54]:
full_activity_ids_order =  full_activity_ids.sort()

In [60]:
full_activity_ids

Out[22]: [3764074685,
 3809768344,
 3865986737,
 3871052555,
 3876332481,
 4079983225,
 4382047703,
 4414021787,
 4730507502,
 4768097716,
 4838912326,
 4910436714,
 4948214694,
 4993537055,
 5198781723,
 5281545880,
 5326544918,
 5391043545,
 5408255638,
 5413547479,
 5448078140,
 5487914339,
 5503770439,
 5508615942,
 5524575370,
 5718648252,
 5740260874,
 5755543494,
 5972584497,
 5987842682,
 5997390802,
 6040654473,
 6059385275,
 6114231607,
 6129061619,
 6162281948,
 6194864569,
 6239126619,
 6261731220,
 6270833770,
 6279812615,
 6284670536,
 6298941266,
 6302328393,
 6395162679,
 6439530045,
 6627964856,
 6648045580,
 6658996209,
 6754060843,
 6759396023,
 6825624837,
 6856368170,
 6886523125,
 6908050193,
 6967425665,
 6967590959,
 6991844521,
 6992488859,
 7004837267,
 7041321207,
 7062090532,
 7095425818,
 7138047360,
 7166250297,
 7208614563,
 7218381768,
 7224011708,
 7229384684,
 7261304130,
 7303733207,
 7358811380,
 7379581860,
 7390526948,
 7399680604,
 7405131726,
 74

In [49]:
try:
    #Query path, see if there are any activities with their associated segments written to storage
    segments_in_storage = spark.read.format("delta").load(segment_effort_path)
except:
    #if that errors, meaning first time running the script
    #Write the first 99 activites to storage, will need to specificy sort order 
    top_99_activity_ids = 
    write_dataframe_to_storage(historical_df_to_write,storagepath, "mergeSchema", "append" )


IllegalArgumentException : java.net.URISyntaxException: Illegal character in scheme name at index 0: [REDACTED]

Error: ---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
<command--1> in <module>
----> 1 segments_in_storage = spark.read.format("delta").load(segment_effort_path)

/databricks/spark/python/pyspark/sql/readwriter.py in load(self, path, format, schema, **options)
    156         self.options(**options)
    157         if isinstance(path, str):
--> 158             return self._df(self._jreader.load(path))
    159         elif path is not None:
    160             if type(path) != list:

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    121                 # Hide wher

In [38]:
segment_id_df.show()

NameError : name 'segment_id_df' is not defined

Error: ---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command--1> in <module>
----> 1 segment_id_df.show()

NameError: name 'segment_id_df' is not defined

Need to limit to 100 request as the api throws an errors after

In [36]:
segment_id_df = query_segments(full_activity_ids)

KeyError : 'segment_efforts'

Error: ---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<command--1> in <module>
----> 1 segment_id_df = query_segments(full_activity_ids)

<command--1> in query_segments(activity_ids)

KeyError: 'segment_efforts'

In [ ]:
write_dataframe_to_storage(segment_id_df,segment_effort_path, "overwriteSchema","overwrite" )

In [ ]:
segments_in_storage = spark.read.format("delta").load(segment_effort_path)

In [ ]:
#hopefull.groupBy('activity_id').agg(countDistinct('segment_id')).show()

In [ ]:
activity_list = hopefull.select('activity_id').distinct().collect()
len(activity_list)
len(subset)

In [ ]:
#need to get distinct activity_ids and run them through the segment
#we make 1 api query in the first script, so gonna be allowed 99 with this run 

In [ ]:
#get all of the activity_ids, limit to 15 as thats how many we can run in a single query
#also need to query the activities we have written to segment storage, so as not to repeat 
#Strava API usage is limited on a per-application basis using both a 15-minute and daily request limit. The default rate limit allows 100 requests every 15 minutes, 
# with up to 1,000 requests per day.

#compare current activites vs what is written, 
#going to need to write some try and excepts for expecting these values in return 